In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def process(edl_constructor, dataframe: pd.DataFrame, pzc_she: float):
    """
    Process Monteiro data and calculate phi0-phi'
    """
    dataframe["E_SHE"] = dataframe["E vs. RHE"] - dataframe["pH"] * 59e-3
    dataframe["phi0"] = dataframe["E_SHE"] - pzc_she

    # Find unique log concentration values
    unique_logc_values = dataframe.drop_duplicates(subset="log c")["log c"].values

    # One computation for each concentration value
    processed_dfs = []

    for logc in unique_logc_values:
        df_select_logc = dataframe[dataframe["log c"] == logc].copy()
        model = edl_constructor(10**logc)

        potentials_vs_pzc = np.linspace(np.min(df_select_logc["phi0"]), 0, 50)
        sweep = model.potential_sweep(potentials_vs_pzc, tol=1e-4)

        # Interpolate the result for the phi0 values already in the dataframe
        interpolator = interp1d(potentials_vs_pzc, sweep["phi_rp"])
        df_select_logc["phi_rp"] = interpolator(df_select_logc["phi0"])
        processed_dfs.append(df_select_logc)

    return_df = pd.concat(processed_dfs)
    return_df['c'] = 10 ** return_df['log c'].values
    return return_df

In [ ]:
from edl.models import ExplicitStern as model1
from edl.models import DoubleLayerModel as model2
from edl.constants import AU_PZC_SHE_V as PZC
species_dict = {'Li': 7, 'Na': 6, 'K': 5}
x2_dict = {'Cs': 3.5e-10, 'K': 4.1e-10, 'Na': 5.2e-10, 'Li': 5.8e-10}

In [ ]:
df_dict = {}
for species, gamma in species_dict.items():
    df = pd.read_csv(f"data/{species}.csv")
    # df = process(lambda c: model1(c, properties[0], 2, x2_dict[species]), df, pzc_she=PZC)
    df = process(lambda c: model2(c, gamma, 2), df, pzc_she=PZC)
    df_dict[species] = df

df_dict_ringe = {}
for species, gamma in species_dict.items():
    df = pd.read_csv(f"data/{species}.csv")
    df = process(lambda c: model1(c, gamma, 2, x2_dict[species]), df, pzc_she=PZC)
    # df = process(lambda c: model2(c, properties[0], 2), df, pzc_she=PZC)
    df_dict_ringe[species] = df

In [ ]:
for species, df in df_dict.items():
    df['species'] = species
all_df = pd.concat(list(df_dict.values()))

for species, df in df_dict_ringe.items():
    df['species'] = species
all_df_ringe = pd.concat(list(df_dict_ringe.values()))

# import pandas as pd
all_df.to_csv('data/au_df.csv', index=False, float_format='%10.3f')
all_df_ringe.to_csv('data/au_df_ringe.csv', index=False, float_format='%10.3f')